In [1]:
from MLimports2 import *
exec(MLimports())


from copy import copy
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from lightgbm import LGBMClassifier as LGB
from scipy import stats
from sklearn import metrics,neighbors,preprocessing
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression as LR
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.me

In [2]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true[np.nonzero(y_true)] # avoid incidents with zero duration
    y_pred = y_pred[np.nonzero(y_true)]
    return abs(np.mean(np.abs((y_true - y_pred) / y_true)) * 100)

def mse(yt,yp):
    return mean_squared_error(yt,yp)

def rmse(yt,yp):
    return np.sqrt(mean_squared_error(yt,yp))

In [16]:
METRIC = []

METRIC.append({'name':'RMSE','instance':rmse})
METRIC.append({'name':'MAPE','instance':mape})

# Eval

In [17]:
from scipy.stats import wasserstein_distance as WD
from sklearn.preprocessing import QuantileTransformer as QT

def EVAL2(REG,MET,setup='AlltoAll',TH=40,log=False,dataset='sf',qtran=False):
    global RESULTS,X,Y
    from sklearn.model_selection import KFold
    import multiprocessing
    
    X, Y = getBFS(part=[0,-1],dataset=dataset)

    X_A = X[Y<TH]
    Y_A = Y[Y<TH]
    
    X_B = X[Y>=TH]
    Y_B = Y[Y>=TH]
    
    search = RandomizedSearchCV(estimator = REG['instance'](),
                       param_distributions=REG['param'],
                       n_iter=250,
                       scoring=make_scorer(MET['instance'], greater_is_better=False),
                       n_jobs=multiprocessing.cpu_count(), ## MAR: used to be 1
                       cv = 10,
                       verbose = 1
                           )
    qt=None
    qY=None
    
    if log:
        search.fit(X,np.log1p(Y))
    elif qtran:
        qt = QT()
        search.fit(X, qt.fit_transform(Y.reshape(-1,1)).ravel() )
    else:
        search.fit(X, Y)
    

    FOLDS=10
    ATTEMPTS=1
    
    SCOREMSE=[]
    SCORER2=[]
    SCORER2ABS=[]
    SCOREMAPE=[]
    
    import tqdm
    
    
    if setup=='AlltoA':
        for A in range(ATTEMPTS):
            
            kf = KFold(n_splits=FOLDS,shuffle=True)

            if setup=='AlltoA':
                print('split AlltoA')
                kf.get_n_splits(X_A)
            
            for train_index, test_index in tqdm.tqdm(kf.split(X_A)):
                
                Xtr, Xte = pd.concat([X_B,X_A.iloc[train_index]], axis=0), X_A.iloc[test_index]
                Ytr, Yte = np.concatenate([np.array(Y_B), np.array(Y_A)[train_index]],axis=0), Y_A[test_index]
                reg = REG['instance'](**search.best_params_)
                if log:
                    reg.fit(Xtr,np.log1p(Ytr))
                elif qtran:
                    qt = QT()
                    reg.fit(Xtr, qt.fit_transform(Ytr.reshape(-1,1)).ravel())
                else:
                    reg.fit(Xtr, Ytr)
                    
                pred = reg.predict(Xte)
                
                if log:
                    pred = np.nan_to_num(np.expm1(pred),0) ###
                
                if qtran:
                    pred = qt.inverse_transform(pred.reshape(-1,1)).ravel()
                
                PRED = np.array(pred).ravel()
                SCOREMSE.append(mse(Yte, PRED))
                SCOREMAPE.append(mape(Yte, PRED))
                
    if setup=='AlltoB':
        for A in range(ATTEMPTS):
            
            kf = KFold(n_splits=FOLDS,shuffle=True)

            if setup=='AlltoB':
                print('split AlltoB')
                kf.get_n_splits(X_B)

                
            for train_index, test_index in tqdm.tqdm(kf.split(X_B)):
                
                Xtr, Xte = pd.concat([X_A,X_B.iloc[train_index]], axis=0), X_B.iloc[test_index]
                Ytr, Yte = np.concatenate([np.array(Y_A), np.array(Y_B)[train_index]],axis=0), Y_B[test_index]
                reg = REG['instance'](**search.best_params_)
                if log:
                    reg.fit(Xtr,np.log1p(Ytr))
                elif qtran:
                    qt = QT()
                    reg.fit(Xtr, qt.fit_transform(Ytr.reshape(-1,1)).ravel())
                else:
                    reg.fit(Xtr, Ytr)
                    
                pred = reg.predict(Xte)
                
                if log:
                    pred = np.nan_to_num(np.expm1(pred),0) ###
                
                if qtran:
                    pred = qt.inverse_transform(pred.reshape(-1,1)).ravel()
                PRED = np.array(pred).ravel()
                SCOREMSE.append(mse(Yte, PRED))
                SCOREMAPE.append(mape(Yte, PRED))
                
    if setup=='AtoA':
        for A in range(ATTEMPTS):
            
            kf = KFold(n_splits=FOLDS,shuffle=True)

            if setup=='AtoA':
                kf.get_n_splits(X_A)
                
            for train_index, test_index in tqdm.tqdm(kf.split(X_A)):
                Xtr, Xte = X_A.iloc[train_index], X_A.iloc[test_index]
                Ytr, Yte = Y_A[train_index], Y_A[test_index]
                reg = REG['instance'](**search.best_params_)
                if log:
                    reg.fit(Xtr,np.log1p(Ytr))
                elif qtran:
                    qt = QT()
                    reg.fit(Xtr, qt.fit_transform(Ytr.reshape(-1,1)).ravel())
                else:
                    reg.fit(Xtr, Ytr)
                    
                pred = reg.predict(Xte)
                
                if log:
                    pred = np.nan_to_num(np.expm1(pred),0) ###
                
                if qtran:
                    pred = qt.inverse_transform(pred.reshape(-1,1)).ravel()
                PRED = np.array(pred).ravel()
                SCOREMSE.append(mse(Yte, PRED))
                SCOREMAPE.append(mape(Yte, PRED))
                
                
    if setup=='AtoB':
        for A in range(ATTEMPTS):
            
            kf = KFold(n_splits=FOLDS,shuffle=True)

            if setup=='AtoB':
                kf.get_n_splits(X_A)
            
            for train_index, test_index in tqdm.tqdm(kf.split(X_A)):
                Xtr, Xte = X_A.iloc[train_index], X_B
                Ytr, Yte = Y_A[train_index], Y_B
                reg = REG['instance'](**search.best_params_)
                if log:
                    reg.fit(Xtr,np.log1p(Ytr))
                elif qtran:
                    qt = QT()
                    reg.fit(Xtr, qt.fit_transform(Ytr.reshape(-1,1)).ravel())
                else:
                    reg.fit(Xtr, Ytr)
                    
                pred = reg.predict(Xte)
                
                if log:
                    pred = np.nan_to_num(np.expm1(pred),0) ###
                
                if qtran:
                    pred = qt.inverse_transform(pred.reshape(-1,1)).ravel()
                PRED = np.array(pred).ravel()
                SCOREMSE.append(mse(Yte, PRED))
                SCOREMAPE.append(mape(Yte, PRED))
                
    if setup=='BtoA':
        print('folding BtoA')
        for A in range(ATTEMPTS):
            
            
            kf = KFold(n_splits=FOLDS,shuffle=True)

            if setup=='BtoA':
                print('split BtoA')
                kf.get_n_splits(X_B)
                
                
            for train_index, test_index in tqdm.tqdm(kf.split(X_B)):
                Xtr, Xte = X_B.iloc[train_index], X_A
                Ytr, Yte = Y_B[train_index], Y_A
                reg = REG['instance'](**search.best_params_)
                if log:
                    reg.fit(Xtr,np.log1p(Ytr))
                elif qtran:
                    qt = QT()
                    reg.fit(Xtr, qt.fit_transform(Ytr.reshape(-1,1)).ravel())
                else:
                    reg.fit(Xtr, Ytr)
                    
                pred = reg.predict(Xte)
                
                if log:
                    pred = np.nan_to_num(np.expm1(pred),0) ###
                
                if qtran:
                    pred = qt.inverse_transform(pred.reshape(-1,1)).ravel()
                PRED = np.array(pred).ravel()
                SCOREMSE.append(mse(Yte, PRED))
                SCOREMAPE.append(mape(Yte, PRED))
    
    if setup=='BtoB':
        print('folding BtoB')
        for A in range(ATTEMPTS):
            
            
            kf = KFold(n_splits=FOLDS,shuffle=True)
    
            if setup=='BtoB':
                print('split BtoB')
                kf.get_n_splits(X_B)
                
            for train_index, test_index in tqdm.tqdm(kf.split(X_B)):
                Xtr, Xte = X_B.iloc[train_index], X_B.iloc[test_index]
                Ytr, Yte = Y_B[train_index], Y_B[test_index]
                reg = REG['instance'](**search.best_params_)
                if log:
                    reg.fit(Xtr,np.log1p(Ytr))
                elif qtran:
                    qt = QT()
                    reg.fit(Xtr, qt.fit_transform(Ytr.reshape(-1,1)).ravel())
                else:
                    reg.fit(Xtr, Ytr)
                    
                pred = reg.predict(Xte)
                
                if log:
                    pred = np.nan_to_num(np.expm1(pred),0) ###
                
                if qtran:
                    pred = qt.inverse_transform(pred.reshape(-1,1)).ravel()
                PRED = np.array(pred).ravel()
                SCOREMSE.append(mse(Yte, PRED))
                SCOREMAPE.append(mape(Yte, PRED))

    
    if setup=='AlltoAll':
        for A in range(ATTEMPTS):
            
            kf = KFold(n_splits=FOLDS,shuffle=True)
    
            if setup=='AlltoAll':
                kf.get_n_splits(X)

            for train_index, test_index in tqdm.tqdm(kf.split(X)):
                Xtr, Xte = X.iloc[train_index], X.iloc[test_index]
                Ytr, Yte = Y[train_index], Y[test_index]
                
#                 Ytr=np.log1p(Ytr) ###
                reg = REG['instance'](**search.best_params_)
    
                if log:
                    reg.fit(Xtr,np.log1p(Ytr))
                elif qtran:
                    qt = QT()
                    reg.fit(Xtr, qt.fit_transform(Ytr.reshape(-1,1)).ravel())
                else:
                    reg.fit(Xtr, Ytr)
                    
                pred = reg.predict(Xte)
                
                if log:
                    pred = np.nan_to_num(np.expm1(pred),0) ###
                
                if qtran:
                    pred = qt.inverse_transform(pred.reshape(-1,1)).ravel()
                PRED = np.array(pred).ravel()
                SCOREMSE.append(mse(Yte, PRED))
                SCOREMAPE.append(mape(Yte, PRED))

    
    for zz in range(len(SCOREMAPE)):
        RESULTS.append({'Optimizer': REG['name'],
          'Training': MET['name'],
          'mape': SCOREMAPE[zz],
          'mse': SCOREMSE[zz],'folding':'KF'})



In [18]:
from sklearn.svm import SVR
import lightgbm as lgb
from sklearn.neighbors import KNeighborsRegressor
OPTIMIZER = []

from sklearn import linear_model                

OPTIMIZER.append({'name':'XGB','instance':XGBRegressor,'param':{
            'learning_rate' : np.linspace(0.0001,0.9999,10000), 
            'n_estimators' : range(20,200,1),'objective':['reg:squarederror','reg:gamma', 'reg:tweedie'],
            'max_depth':range(2,15,1), ##MAR: start at 3 
            'subsample':np.linspace(0.3, 0.9 ,10000),'colsample_bytree': np.linspace(0.1,0.9,10000),
            'min_child_weight':np.arange(2,15),
#             'gamma':np.arange(0.01,5),
#             'reg_alpha':np.arange(0,5),'reg_lambda':np.arange(0,5)
        }})

OPTIMIZER.append({'name':'LGBM','instance':lgb.LGBMRegressor,'param':{
            'feature_fraction':np.linspace(0.5,1,100),'bagging_fraction':np.linspace(0.5,1,100),
            'learning_rate' : np.linspace(0.00001,1,10000), 'lambda_l1':np.linspace(0,5,1000),'lambda_l2':np.linspace(0,5,1000),
            'max_depth' : np.arange(2,15), 'metric':['MAE','RMSE'],'objective':['huber', 'gamma', 'fair', 'tweedie'],
        }})

OPTIMIZER.append({'name':'KNN','instance':KNeighborsRegressor,'param':{
            'n_neighbors' : range(2,100), 
            'weights' : ['uniform','distance'],
            'p':[1,2], 'leaf_size': np.arange(2,60)
        }})

# OPTIMIZER.append({'name':'CB','instance':CatBoostRegressor,'param':{
#             'learning_rate' : np.linspace(0.0001,1,10000),'n_estimators':np.arange(20,200),
#             'depth' : np.arange(2,9), 'verbose': [0], 'l2_leaf_reg': np.linspace(0,5,1000), 'eval_metric':['MAE', 'RMSE', 'Poisson'],
# #             'p':[1,2], 'leaf_size': np.arange(2,60)
#         }})

# OPTIMIZER.append({'name':'NGB','instance': NGBRegressor, 'param':{
#     'learning_rate' : np.linspace(0.0001,1,10000),
# #     'minibatch_frac' : np.linspace(0.001,1,1000), 
#     'n_estimators' : range(20,250,1),
# #     'max_depth':range(2,15,1), ##MAR: start at 3 
# #     'subsample':np.linspace(0.25, 1.0 ,10000), 'min_samples_split':np.linspace(0.001,0.999,1000)
# }})

OPTIMIZER.append({'name':'GBDT','instance': GradientBoostingRegressor, 'param':{
    'learning_rate' : np.linspace(0.00001,1,10000), 
    'n_estimators' : range(20,200,1),
    'max_depth':range(2,15,1), ##MAR: start at 3 
    'subsample':np.linspace(0.25, 1.0 ,10000), 'min_samples_split':np.linspace(0.001,0.999,1000)
}})

OPTIMIZER.append({'name':'LR','instance': Ridge, 'param':{
    'alpha':np.linspace(0,10,10000), 'normalize': [True,False], 'fit_intercept':[True,False]
}})

OPTIMIZER.append({'name':'BR','instance':linear_model.BayesianRidge,'param':{
            'alpha_1':np.linspace(1e-6,1,10000),
            'alpha_2':np.linspace(1e-6,1,10000),
            'lambda_1':np.linspace(1e-6,1,10000),
            'lambda_2':np.linspace(1e-6,1,10000),
        }})               

OPTIMIZER.append({'name':'RF','instance': RandomForestRegressor, 'param':{
    'n_estimators' : range(20,200,1),
    'max_depth':range(2,15,1), ##MAR: start at 3 
}})

In [19]:
X,Y = getBFS(part=[0,-1],dataset='sf')

RESULTS = []
for experiment in ['AtoA','AtoB','BtoA','AlltoA','AlltoB','AlltoAll']:
    import tqdm
    for O in tqdm.tqdm(OPTIMIZER):
        print(O['name'])
        for M in METRIC:
            print(M['name'])
            print('lpo')
            #EVAL2 populates RESULTS list
            EVAL2(O,M,setup=experiment,dataset='sf',TH=40,log=False,qtran=False)

  0%|          | 0/7 [00:00<?, ?it/s]

XGB
RMSE
lpo
Fitting 10 folds for each of 250 candidates, totalling 2500 fits


[Parallel(n_jobs=56)]: Using backend LokyBackend with 56 concurrent workers.
[Parallel(n_jobs=56)]: Done  88 tasks      | elapsed:   12.9s
[Parallel(n_jobs=56)]: Done 338 tasks      | elapsed:   13.6s
[Parallel(n_jobs=56)]: Done 688 tasks      | elapsed:   14.9s
[Parallel(n_jobs=56)]: Done 1138 tasks      | elapsed:   16.3s
[Parallel(n_jobs=56)]: Done 1688 tasks      | elapsed:   17.9s
[Parallel(n_jobs=56)]: Done 2338 tasks      | elapsed:   19.9s
[Parallel(n_jobs=56)]: Done 2500 out of 2500 | elapsed:   20.4s finished
/home/agrigore/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)

0it [00:00, ?it/s]
2it [00:00, 16.24it/s]
5it [00:00, 17.75it/s]
7it [00:00, 14.57it/s]
10it [00:00, 13.51it/s][A
[Parallel(n_jobs=56)]: Using backend LokyBackend

kfold
MAPE
lpo
Fitting 10 folds for each of 250 candidates, totalling 2500 fits


[Parallel(n_jobs=56)]: Done  88 tasks      | elapsed:    0.5s
[Parallel(n_jobs=56)]: Done 338 tasks      | elapsed:    1.2s
[Parallel(n_jobs=56)]: Done 688 tasks      | elapsed:    2.2s
[Parallel(n_jobs=56)]: Done 1138 tasks      | elapsed:    3.6s
[Parallel(n_jobs=56)]: Done 1688 tasks      | elapsed:    5.3s
[Parallel(n_jobs=56)]: Done 2338 tasks      | elapsed:    7.1s
[Parallel(n_jobs=56)]: Done 2500 out of 2500 | elapsed:    7.6s finished
/home/agrigore/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)

0it [00:00, ?it/s]
3it [00:00, 28.55it/s]
10it [00:00, 37.87it/s][A
[Parallel(n_jobs=56)]: Using backend LokyBackend with 56 concurrent workers.


...

kfold
SMAPE
lpo
Fitting 10 folds for each of 250 candidates, totalling 2500 fits


[Parallel(n_jobs=56)]: Done  88 tasks      | elapsed:    0.7s
[Parallel(n_jobs=56)]: Done 338 tasks      | elapsed:    2.5s
[Parallel(n_jobs=56)]: Done 688 tasks      | elapsed:    5.3s
[Parallel(n_jobs=56)]: Done 1138 tasks      | elapsed:    8.3s
[Parallel(n_jobs=56)]: Done 1688 tasks      | elapsed:   11.7s
[Parallel(n_jobs=56)]: Done 2338 tasks      | elapsed:   15.7s
[Parallel(n_jobs=56)]: Done 2500 out of 2500 | elapsed:   16.9s finished
/home/agrigore/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)

0it [00:00, ?it/s]
1it [00:00,  7.74it/s]
2it [00:00,  7.87it/s]
3it [00:00,  7.91it/s]
4it [00:00,  7.86it/s]
5it [00:00,  7.83it/s]
6it [00:00,  7.91it/s]
7it [00:00,  7.84it/s]
8it [00:01,  7.76it/s]
9it [00:01,  7.76it/s]
10it [00:01, 

kfold
logWD
lpo
Fitting 10 folds for each of 250 candidates, totalling 2500 fits


[Parallel(n_jobs=56)]: Done  88 tasks      | elapsed:    0.9s
[Parallel(n_jobs=56)]: Done 564 tasks      | elapsed:    3.8s
[Parallel(n_jobs=56)]: Done 1020 tasks      | elapsed:    6.7s
[Parallel(n_jobs=56)]: Done 1470 tasks      | elapsed:    9.4s
[Parallel(n_jobs=56)]: Done 2020 tasks      | elapsed:   12.7s
[Parallel(n_jobs=56)]: Done 2500 out of 2500 | elapsed:   15.9s finished
/home/agrigore/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)

0it [00:00, ?it/s]
2it [00:00, 18.10it/s]
4it [00:00, 18.21it/s]
6it [00:00, 18.15it/s]
8it [00:00, 18.18it/s]
10it [00:00, 18.23it/s]
100%|██████████| 7/7 [03:36<00:00, 30.90s/it]

kfold


In [20]:
RESULTS
RES = pd.DataFrame(RESULTS)
RES.to_csv("extrapolation.csv",index=false)

,Optimizer,Training,mape,smape,csmape,r2,r2abs,mse,WD,folding
0,XGB,RMSE,65.257056,46.831741,63.611916,-0.045706,0.045706,140.452420,6.160600,KF
1,XGB,RMSE,52.956125,38.097111,47.899022,-0.022714,0.022714,98.325785,5.161770,KF
2,XGB,RMSE,72.759959,48.919465,71.457445,-0.204415,0.204415,159.873529,6.903147,KF
3,XGB,RMSE,90.419278,59.202514,88.601153,-0.412441,0.412441,218.738070,6.934836,KF
4,XGB,RMSE,54.548364,41.717375,51.096861,0.005795,0.005795,97.675589,5.699479,KF
...,...,...,...,...,...,...,...,...,...,...
345,RF,logWD,85.060817,51.049506,83.294951,-0.149792,0.149792,145.187343,6.996993,KF
346,RF,logWD,66.615789,42.844265,61.745165,0.074288,0.074288,114.912266,7.098569,KF
347,RF,logWD,50.977088,39.572994,49.685731,0.024723,0.024723,115.159094,6.360667,KF
348,RF,logWD,64.249472,41.374748,62.515225,-0.085491,0.085491,113.218974,6.281806,KF
